In [1]:
import logging
from pathlib import Path
import pandas as pd
from ingestion.db_utils import (
    fetch_latest_table_data,
    insert_variable,
    load_report_params,
)
from reporting.quarterly_report.utils import RenderContext, BaseModule
from ingestion.db_utils import load_report_params
from typing import Tuple

# our project
from ingestion.db_utils import (
    init_db,                                 # create tables if missing
    fetch_latest_table_data,                 # new version!
    get_alias_last_load,
    get_variable_status, 
    load_report_params                   # to inspect results
)

from reporting.quarterly_report.utils import Database, RenderContext


# ──────────────────────────────────────────────────────────────
# COSTANTS
# ──────────────────────────────────────────────────────────────
R_MONITORING = "reinforced_monitoring"

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('edes.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("Edes Monitoring")



db_path = "database/reporting.db"
DB_PATH = Path("database/reporting.db")



db = Database(str(DB_PATH))         # thin sqlite3 wrapper
conn = db.conn
report = 'Quarterly_Report'

CALLS_TYPES_LIST = ['STG', 'ADG', 'POC', 'COG', 'SYG', 'StG', 'CoG', 'AdG', 'SyG', 'PoC', 'CSA']


# ──────────────────────────────────────────────────────────────
# HELPERS
# ──────────────────────────────────────────────────────────────

def get_scope_start_end(cutoff: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """
    Unified scope logic with year transition:
    • If cutoff is in January → report full previous year
    • Otherwise → return start of year to quarter-end
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        return pd.Timestamp(year=year, month=1, day=1), pd.Timestamp(year=year, month=12, day=31)

    def quarter_end(cutoff: pd.Timestamp) -> pd.Timestamp:
        first_day = cutoff.replace(day=1)
        last_month = first_day - pd.offsets.MonthBegin()
        m = last_month.month

        if m <= 3:
            return pd.Timestamp(year=cutoff.year, month=3, day=31)
        elif m <= 6:
            return pd.Timestamp(year=cutoff.year, month=6, day=30)
        elif m <= 9:
            return pd.Timestamp(year=cutoff.year, month=9, day=30)
        else:
            return pd.Timestamp(year=cutoff.year, month=12, day=31)

    return pd.Timestamp(year=cutoff.year, month=1, day=1), quarter_end(cutoff)


def determine_call_type(row):

    call = str(row.get('CALL', '')).strip()

    try:
        call and any(call_type in call for call_type in CALLS_TYPES_LIST)
        category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in call).upper()
        return category
    except Exception as e:
        raise



In [101]:
EDES_ALIAS = "edes_warnings"


cutoff = pd.to_datetime("2025-06-01")
report_params = load_report_params(report_name=report, db_path=db_path)


table_colors = report_params.get('TABLE_COLORS', {})
BLUE = table_colors.get("BLUE", "#004A99")
LIGHT_BLUE = table_colors.get("LIGHT_BLUE", "#d6e6f4")
DARK_BLUE = table_colors.get("DARK_BLUE", "#01244B")
SUB_TOTAL_BACKGROUND = table_colors.get("subtotal_background_color", "#E6E6FA")

df_edes = fetch_latest_table_data(conn, EDES_ALIAS, cutoff)

df_edes['VALID_FROM'] = pd.to_datetime(
    df_edes['VALID_FROM'], 
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

quarter_dates = get_scope_start_end(cutoff=cutoff)
last_valid_date = quarter_dates[1]

df_edes = df_edes[
    df_edes['VALID_FROM'] <= last_valid_date
].copy()

df_edes['COUNTER'] = 1
df_edes['CALL_TYPE'] = df_edes.apply(determine_call_type, axis=1)


DEBUG:root:Fetching latest data for table_alias: edes_warnings, cutoff: 2025-06-01T00:00:00
DEBUG:root:Upload log query results for edes_warnings: [('2025-06-16T06:47:03.220726', 1)]
DEBUG:root:Checking upload_id: 1, uploaded_at: 2025-06-16T06:47:03.220726
DEBUG:root:Fetched 4 rows from edes_warnings with upload_id 1


In [102]:
edes_pivot = pd.pivot_table(df_edes, values='COUNTER', index=['UNIT'],
                       columns=['CALL_TYPE'],  fill_value=0, aggfunc="sum")

# Drop MultiIndex if needed
if isinstance(edes_pivot.columns, pd.MultiIndex):
        edes_pivot.columns = edes_pivot.columns.rename_axis(None, axis=1).reset_index(drop=True)


edes_pivot.reset_index(inplace=True)
edes_pivot


CALL_TYPE,UNIT,POC,STG
0,ERCEA/C/01,0,1
1,ERCEA/C/02,2,0
2,ERCEA/C/03,0,1


In [99]:
from great_tables import GT, exibble, md, style, loc

def format_edes_table(df, cutoff=cutoff):
    
    year_value = cutoff.year
    stub_width=250

    # Calculate table width
    base_width_per_column = 80
    data_columns = df.columns[1:].tolist()
    table_width = f"{stub_width + (len(data_columns) * base_width_per_column)}px"
    table_width_px = stub_width + (len(data_columns) * base_width_per_column)

    # Payment table specific height components
    
    title_height = 30        # Main title
    subtitle_height = 20     # Subtitle
    column_header_height = 35  # Column headers
    row_height = 40          # Each data row (payment tables tend to be compact)
    footer_padding = 30      # Bottom padding
    border_padding = 20      # Extra space for borders and margins

    # Height calculation (NEW)
    num_rows = len(df)
    
    # Calculate total height
    total_header_height = title_height + subtitle_height + column_header_height
    total_data_height = num_rows * row_height
    table_height_px = total_header_height + total_data_height + footer_padding + border_padding
    # Payment table adjustments
    # Add extra height if we have deviation rows (they might need more space)
    deviation_rows = sum(1 for idx, row in df.iterrows() 
                    if 'deviation' in str(row.iloc[0]).lower())
    if deviation_rows > 0:
        table_height_px += deviation_rows * 5  # Small extra height for colored cells
    
    # Safety margins - ensure minimum and maximum bounds
    table_height_px = max(300, min(table_height_px, 1200))  # Min 300px, Max 1200px
    table_width_px = max(600, min(table_width_px, 1800))    # Min 600px, Max 1800px
                    
    try:
        gt_ex = (
            GT(edes_pivot, rowname_col="UNIT")
            .tab_header(
                title=md(f"Results of the **EDES** screening in **{year_value}**"),
                subtitle=md("This is a breakdown by Unit."),
            )
            .opt_table_font(font="Arial")
            .opt_stylize(style=3, color="blue")
            .tab_style(
                style.text(color=DARK_BLUE, align="center"),
                locations=loc.header()
            )
            .tab_stubhead(label="UNIT")
            .tab_options(
                table_width=table_width
            )
            .tab_source_note(source_note="Source Data: Compass/Sygma")
            .opt_vertical_padding(scale=0.5)
        )
        return gt_ex, table_width_px, table_height_px
    except Exception as e:
        return (f"Error processing: {str(e)}")



var_name = 'EDES_Table'
logger.debug(f"Creating {var_name}")
try:
    tbl,table_width_px, table_height_px = format_edes_table(edes_pivot, cutoff)
except Exception as format_error:
    print (f"      ❌ Error formatting table for Edes: {str(format_error)}")

try:    
    logger.debug(f"Saving {var_name} to database")
    # Save chart
    insert_variable(
        report=report, 
        module="EdesModule", 
        var=var_name,
        value=edes_pivot,
        db_path=db_path, 
        anchor=var_name, 
        simple_gt_save= True,
        gt_table=tbl,
        table_width=table_width_px,      # Automatically calculated
        table_height=table_height_px ,    # Automatically calculated  
    )
except Exception as save_error:
    print(f'❌ Failed to save Table Edes: {str(save_error)}')



DEBUG:Edes Monitoring:Creating EDES_Table
DEBUG:Edes Monitoring:Saving EDES_Table to database
DEBUG:root:Rendering gt_table for EDES_Table
INFO:root:Saving GT table EDES_Table with dimensions 600x300
DEBUG:selenium.webdriver.common.selenium_manager:Selenium Manager binary found at: /usr/local/python/3.12.1/lib/python3.12/site-packages/selenium/webdriver/common/linux/selenium-manager
DEBUG:selenium.webdriver.common.selenium_manager:Executing process: /usr/local/python/3.12.1/lib/python3.12/site-packages/selenium/webdriver/common/linux/selenium-manager --browser chrome --debug --language-binding python --output json
DEBUG:selenium.webdriver.common.selenium_manager:chromedriver not found in PATH
DEBUG:selenium.webdriver.common.selenium_manager:chrome not found in PATH
DEBUG:selenium.webdriver.common.selenium_manager:chrome not found in the system
DEBUG:selenium.webdriver.common.selenium_manager:Required browser: chrome 137.0.7151.70
DEBUG:selenium.webdriver.common.selenium_manager:chrome 